# Trading & Positions

[index](./index.ipynb) |
[accounts](./accounts.ipynb) |
[balances](./balances.ipynb) |
[instruments](./instrumentlookup.ipynb)

In [1]:
from saxo_openapi import API
import saxo_openapi.endpoints.trading as tr
import saxo_openapi.endpoints.portfolio as pf
import json
import exampleauth

token = exampleauth.read_token()

In [2]:
# The client to process the requests
client = API(access_token=token)

## Current Positions

In [3]:
r = pf.positions.PositionsMe()
rv = client.request(r)
print(json.dumps(rv, indent=2))

{
  "__count": 2,
  "Data": [
    {
      "NetPositionId": "EURUSD__43551NY__1.133__C",
      "PositionBase": {
        "AccountId": "9300675",
        "Amount": -100000.0,
        "AssetType": "FxVanillaOption",
        "CanBeClosed": true,
        "ClientId": "9300675",
        "CloseConversionRateSettled": false,
        "CorrelationKey": "69f5c547-92eb-48bf-8130-1249e388c676",
        "ExecutionTimeOpen": "2019-03-14T00:01:03.805780Z",
        "IsMarketOpen": true,
        "OpenPrice": 0.00504,
        "OptionsData": {
          "CanBeExercised": false,
          "ExpiryCut": "NY",
          "ExpiryDate": "2019-03-27T00:00:00.000000Z",
          "PremiumDate": "2019-03-18T00:00:00.000000Z",
          "PutCall": "Call",
          "SettlementStyle": "PhysicalDelivery",
          "Strike": 1.133
        },
        "RelatedOpenOrders": [],
        "SpotDate": "2019-03-29",
        "Status": "Open",
        "Uic": 21,
        "ValueDate": "2019-03-18T00:00:00.000000Z"
      },
      "Po

In [10]:
print("API-endpoint: {}".format(r))
print("METHOD: {}".format(r.method))
print("Response status: {}".format(r.status_code))

API-endpoint: openapi/port/v1/positions/me
METHOD: GET
Response status: 200


## Place some market orders

First retrieve som *account information*.

In [4]:
r = pf.accounts.AccountsMe()
rv = client.request(r)
print(json.dumps(rv, indent=2))

{
  "Data": [
    {
      "AccountGroupKey": "fOA0tvOyQqW2aHpWi9P5bw==",
      "AccountId": "9300675",
      "AccountKey": "fOA0tvOyQqW2aHpWi9P5bw==",
      "AccountType": "Normal",
      "Active": true,
      "CanUseCashPositionsAsMarginCollateral": true,
      "CfdBorrowingCostsActive": false,
      "ClientId": "9300675",
      "ClientKey": "fOA0tvOyQqW2aHpWi9P5bw==",
      "CreationDate": "2019-03-11T11:39:00.000000Z",
      "Currency": "EUR",
      "CurrencyDecimals": 2,
      "DirectMarketAccess": false,
      "IndividualMargining": false,
      "IsCurrencyConversionAtSettlementTime": true,
      "IsMarginTradingAllowed": true,
      "IsShareable": false,
      "IsTrialAccount": true,
      "LegalAssetTypes": [
        "FxSpot",
        "FxForwards",
        "FxVanillaOption",
        "ContractFutures",
        "FuturesOption",
        "Stock",
        "StockOption",
        "CfdOnStock",
        "Bond",
        "MutualFund",
        "CfdOnIndex",
        "CfdOnFutures",
        "

In [5]:
print("API-endpoint: {}".format(r))
print("METHOD: {}".format(r.method))
print("Response status: {}".format(r.status_code))
# process all accounts in Data[]
acct = rv['Data'][0]
for k in ['AccountId', 'AccountKey', 'AccountGroupKey', 'ClientId', 'ClientKey']: 
    print("{:<20s} : {:s}".format(k, acct[k]))

# for new orders the AccountKey is needed
AccountKey = acct['AccountKey']

API-endpoint: openapi/port/v1/accounts/me
METHOD: GET
Response status: 200
AccountId            : 9300675
AccountKey           : fOA0tvOyQqW2aHpWi9P5bw==
AccountGroupKey      : fOA0tvOyQqW2aHpWi9P5bw==
ClientId             : 9300675
ClientKey            : fOA0tvOyQqW2aHpWi9P5bw==


### Define some orders

Define some market orders by putting *orderbodies* in an array:

In [6]:
# Place some market orders
MO = [
   {
       "AccountKey": AccountKey,
       "Amount": "100000",
       "AssetType": "FxSpot",
       "BuySell": "Sell",
       "OrderType": "Market",
       "Uic": 21   # EURUSD
   },
   {
       "AccountKey": AccountKey,
       "Amount": "80000",
       "AssetType": "FxSpot",
       "BuySell": "Buy",
       "OrderType": "Market",
       "Uic": 23   # GBPCAD
   },
]

# create Order requests and process them
for r in [tr.orders.Order(data=orderspec) for orderspec in MO]:
    client.request(r)

and request the positions again ...

In [7]:
r = pf.positions.PositionsMe()
rv = client.request(r)
print(json.dumps(rv, indent=2))

{
  "__count": 4,
  "Data": [
    {
      "NetPositionId": "GBPCAD__FxSpot",
      "PositionBase": {
        "AccountId": "9300675",
        "Amount": 80000.0,
        "AssetType": "FxSpot",
        "CanBeClosed": true,
        "ClientId": "9300675",
        "CloseConversionRateSettled": false,
        "CorrelationKey": "c56ec696-9194-4dc4-a841-a98ca3b411e1",
        "ExecutionTimeOpen": "2019-03-20T21:24:49.808377Z",
        "IsMarketOpen": true,
        "OpenPrice": 1.75558,
        "RelatedOpenOrders": [],
        "SourceOrderId": "76519420",
        "SpotDate": "2019-03-25",
        "Status": "Open",
        "Uic": 23,
        "ValueDate": "2019-03-25T00:00:00.000000Z"
      },
      "PositionId": "214340861",
      "PositionView": {
        "CalculationReliability": "Ok",
        "ConversionRateCurrent": 0.65865,
        "ConversionRateOpen": 0.65865,
        "CurrentPrice": 1.75393,
        "CurrentPriceDelayMinutes": 0,
        "CurrentPriceType": "Bid",
        "Exposure": 8000

The number of positions went from 2 to 4. The new positions can be found in the *Data* array.